In [1]:
from common import *

In [2]:
fnames = glob('../data/lingvo/raw/*')

In [3]:
import gc, io

# prog = re.compile("[\W\d]", re.UNICODE)

def process(fnames, i):
    with io.open('../data/lingvo/%s.txt'%i, 'w', encoding='utf8') as fw:
        for fn in fnames:
            with io.open(fn, encoding='utf8') as fr:
                text = fr.read()
            sents = sent_tokenize(text)
            sents = [[w for w in s.split() if w not in stop_list and len(w)>1] 
                     for s in sents]
            sents = [s for s in sents if len(s)]
            s = json.dumps((basename(fn).split('.')[0], sents), ensure_ascii=False)            
            fw.write(s + u'\n')
    gc.collect()

In [ ]:
parallelizer = Parallel(n_jobs=cpu_count)

tasks_iterator = ( delayed(process)(list_block, i) for 
                  i, list_block in enumerate(grouper(len(fnames)//500, fnames)) ) 
result = parallelizer( tasks_iterator )

In [ ]:
pwd = !pwd
%cd ../data/lingvo/
!cat *.txt > corpus_json.txt&&gzip -f corpus_json.txt&&rm *.txt
%cd {pwd[0]}

In [ ]:
import ujson

def iter_docs(corpus_path):
    with GzipFile(corpus_path, 'r') as fr:
        for line in fr:
            _id, sents = ujson.loads(line)
            yield _id, sents
    
def iter_sents(corpus_path):
    for _id, sents in iter_docs(corpus_path):
        for s in sents:
            yield s
                
class Sentences(object):
    def __init__(self, corpus_path):
        self.corpus_path = corpus_path
    def __iter__(self):
        for sent in iter_sents(self.corpus_path):
            yield sent
            
            
def extract_bigrams(corpus_path, name, min_count=5, threshold=10):
    bigram_path = '../data/lingvo/%s' % name
    bigram = gensim.models.Phrases(iter_sents(corpus_path),
                                   progress_per=100000)
    bigram.min_count = min_count
    bigram.threshold = threshold

    bigram.save(bigram_path)
    logging.info('vocab size %s' % len(bigram.vocab))
    
    bigram_ph = gensim.models.phrases.Phraser(bigram)
    bigram_ph_path = bigram_path + '_ph_%s_%s' % (min_count, threshold)
    bigram_ph.save(bigram_ph_path)
    
    bigram_corpus_path = '../data/lingvo/%s_corpus.txt.gz' % name
    logging.info('saving %s' % bigram_corpus_path)
    with GzipFile(bigram_corpus_path, 'w') as f:
        for sent in bigram_ph[iter_sents(corpus_path)]:
            f.write(unicode(' '.join(sent)).encode('utf8') + '\n')

# Bigrams

In [ ]:
name = 'bigram'
corpus_path = '../data/lingvo/corpus.txt.gz'
igram_path = '../data/lingvo/%s' % name
min_count, threshold = 20, 30

In [ ]:
extract_bigrams(corpus_path, name, min_count, threshold)

In [15]:
bigram = gensim.models.phrases.Phrases.load(bigram_path)
for phrase, score in bigram.export_phrases(islice(iter_sents(corpus_path), 500)):
    print('{0}   {1}'.format(phrase, score))

торговый наименование   46.0968005278
ti zr   54.2023204961
et al   40.2187126168
gas turbine   1327.44429617
advanced materials   222.224706823
цирконий гафний   40.7752414915
цирконий гафний   40.7752414915
american society   297.733331399
test materials   106.495165158
американский общество   160.174541596
беговой дорожка   82.4931135071
беговой дорожка   82.4931135071
беговой дорожка   82.4931135071
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхронный резонанс   64.1262090751
подсинхр

In [ ]:
bigram_ph_path = bigram_path + '_ph_%s_%s' % (min_count, threshold)
bigram_ph = gensim.models.phrases.Phraser.load(bigram_ph_path)

# Trigrams

In [ ]:
name = 'trigram'
corpus_path = '../data/lingvo/bigram_corpus.txt.gz'
trigram_path = '../data/lingvo/%s' % name
min_count, threshold = 20, 30

In [ ]:
extract_bigrams(corpus_path, name, min_count, threshold)

In [ ]:
trigram_ph_path = bigram_path + '_ph_%s_%s' % (min_count, threshold)
trigram_ph = gensim.models.phrases.Phraser.load(trigram_ph_path)

# Word2Vec

In [10]:
corpus_path = '../data/lingvo/trigram_corpus.txt.gz'
model = Word2Vec(Sentences(corpus_path), size=300, sg=1, 
                 min_count=5, window=10, workers=cpu_count)

2017-10-25 07:34:42,716 [MainThread  ] [INFO ]  collecting all words and their counts
2017-10-25 07:34:42,717 [MainThread  ] [INFO ]  PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-25 07:34:43,721 [MainThread  ] [INFO ]  PROGRESS: at sentence #10000, processed 113858 words, keeping 13569 word types
2017-10-25 07:34:44,690 [MainThread  ] [INFO ]  PROGRESS: at sentence #20000, processed 225483 words, keeping 22380 word types
2017-10-25 07:34:45,590 [MainThread  ] [INFO ]  PROGRESS: at sentence #30000, processed 326531 words, keeping 30608 word types
2017-10-25 07:34:46,508 [MainThread  ] [INFO ]  PROGRESS: at sentence #40000, processed 430515 words, keeping 39716 word types
2017-10-25 07:34:47,375 [MainThread  ] [INFO ]  PROGRESS: at sentence #50000, processed 527838 words, keeping 45677 word types
2017-10-25 07:34:48,360 [MainThread  ] [INFO ]  PROGRESS: at sentence #60000, processed 639470 words, keeping 53030 word types
2017-10-25 07:34:49,383 [MainThread  ]

2017-10-25 07:35:41,835 [MainThread  ] [INFO ]  PROGRESS: at sentence #630000, processed 6626137 words, keeping 323056 word types
2017-10-25 07:35:42,755 [MainThread  ] [INFO ]  PROGRESS: at sentence #640000, processed 6728370 words, keeping 328670 word types
2017-10-25 07:35:43,689 [MainThread  ] [INFO ]  PROGRESS: at sentence #650000, processed 6833990 words, keeping 332649 word types
2017-10-25 07:35:44,602 [MainThread  ] [INFO ]  PROGRESS: at sentence #660000, processed 6935887 words, keeping 336545 word types
2017-10-25 07:35:45,576 [MainThread  ] [INFO ]  PROGRESS: at sentence #670000, processed 7045297 words, keeping 339651 word types
2017-10-25 07:35:46,559 [MainThread  ] [INFO ]  PROGRESS: at sentence #680000, processed 7156996 words, keeping 343200 word types
2017-10-25 07:35:47,560 [MainThread  ] [INFO ]  PROGRESS: at sentence #690000, processed 7270858 words, keeping 346544 word types
2017-10-25 07:35:48,541 [MainThread  ] [INFO ]  PROGRESS: at sentence #700000, processed 7

2017-10-25 07:36:41,004 [MainThread  ] [INFO ]  PROGRESS: at sentence #1260000, processed 13207020 words, keeping 557293 word types
2017-10-25 07:36:41,988 [MainThread  ] [INFO ]  PROGRESS: at sentence #1270000, processed 13316572 words, keeping 560491 word types
2017-10-25 07:36:42,946 [MainThread  ] [INFO ]  PROGRESS: at sentence #1280000, processed 13424982 words, keeping 563859 word types
2017-10-25 07:36:43,919 [MainThread  ] [INFO ]  PROGRESS: at sentence #1290000, processed 13534258 words, keeping 566722 word types
2017-10-25 07:36:44,902 [MainThread  ] [INFO ]  PROGRESS: at sentence #1300000, processed 13641596 words, keeping 569733 word types
2017-10-25 07:36:45,833 [MainThread  ] [INFO ]  PROGRESS: at sentence #1310000, processed 13744300 words, keeping 573588 word types
2017-10-25 07:36:46,764 [MainThread  ] [INFO ]  PROGRESS: at sentence #1320000, processed 13844157 words, keeping 576992 word types
2017-10-25 07:36:47,736 [MainThread  ] [INFO ]  PROGRESS: at sentence #13300

2017-10-25 07:37:40,335 [MainThread  ] [INFO ]  PROGRESS: at sentence #1890000, processed 19795754 words, keeping 763983 word types
2017-10-25 07:37:41,215 [MainThread  ] [INFO ]  PROGRESS: at sentence #1900000, processed 19884586 words, keeping 766590 word types
2017-10-25 07:37:42,154 [MainThread  ] [INFO ]  PROGRESS: at sentence #1910000, processed 19986496 words, keeping 769985 word types
2017-10-25 07:37:43,117 [MainThread  ] [INFO ]  PROGRESS: at sentence #1920000, processed 20094067 words, keeping 772206 word types
2017-10-25 07:37:44,155 [MainThread  ] [INFO ]  PROGRESS: at sentence #1930000, processed 20210156 words, keeping 774963 word types
2017-10-25 07:37:45,052 [MainThread  ] [INFO ]  PROGRESS: at sentence #1940000, processed 20308237 words, keeping 777165 word types
2017-10-25 07:37:45,981 [MainThread  ] [INFO ]  PROGRESS: at sentence #1950000, processed 20410772 words, keeping 779801 word types
2017-10-25 07:37:46,906 [MainThread  ] [INFO ]  PROGRESS: at sentence #19600

2017-10-25 07:38:39,834 [MainThread  ] [INFO ]  PROGRESS: at sentence #2520000, processed 26364114 words, keeping 945315 word types
2017-10-25 07:38:40,784 [MainThread  ] [INFO ]  PROGRESS: at sentence #2530000, processed 26469294 words, keeping 947759 word types
2017-10-25 07:38:41,753 [MainThread  ] [INFO ]  PROGRESS: at sentence #2540000, processed 26578408 words, keeping 949771 word types
2017-10-25 07:38:42,738 [MainThread  ] [INFO ]  PROGRESS: at sentence #2550000, processed 26688664 words, keeping 953212 word types
2017-10-25 07:38:43,684 [MainThread  ] [INFO ]  PROGRESS: at sentence #2560000, processed 26792689 words, keeping 955616 word types
2017-10-25 07:38:44,658 [MainThread  ] [INFO ]  PROGRESS: at sentence #2570000, processed 26902682 words, keeping 957889 word types
2017-10-25 07:38:45,593 [MainThread  ] [INFO ]  PROGRESS: at sentence #2580000, processed 27006267 words, keeping 960122 word types
2017-10-25 07:38:46,510 [MainThread  ] [INFO ]  PROGRESS: at sentence #25900

...

2017-10-25 10:12:33,011 [MainThread  ] [INFO ]  PROGRESS: at sentence #91090000, processed 956248299 words, keeping 11198746 word types
2017-10-25 10:12:34,170 [MainThread  ] [INFO ]  PROGRESS: at sentence #91100000, processed 956353935 words, keeping 11199409 word types
2017-10-25 10:12:35,361 [MainThread  ] [INFO ]  PROGRESS: at sentence #91110000, processed 956464656 words, keeping 11200237 word types
2017-10-25 10:12:36,546 [MainThread  ] [INFO ]  PROGRESS: at sentence #91120000, processed 956572975 words, keeping 11201167 word types
2017-10-25 10:12:37,580 [MainThread  ] [INFO ]  PROGRESS: at sentence #91130000, processed 956661409 words, keeping 11202600 word types
2017-10-25 10:12:38,738 [MainThread  ] [INFO ]  PROGRESS: at sentence #91140000, processed 956769940 words, keeping 11203067 word types
2017-10-25 10:12:39,902 [MainThread  ] [INFO ]  PROGRESS: at sentence #91150000, processed 956877288 words, keeping 11204873 word types
2017-10-25 10:12:41,066 [MainThread  ] [INFO ]  

2017-10-25 10:13:42,867 [MainThread  ] [INFO ]  PROGRESS: at sentence #91700000, processed 962651907 words, keeping 11246888 word types
2017-10-25 10:13:43,955 [MainThread  ] [INFO ]  PROGRESS: at sentence #91710000, processed 962751091 words, keeping 11247612 word types
2017-10-25 10:13:45,083 [MainThread  ] [INFO ]  PROGRESS: at sentence #91720000, processed 962854764 words, keeping 11248228 word types
2017-10-25 10:13:46,257 [MainThread  ] [INFO ]  PROGRESS: at sentence #91730000, processed 962959640 words, keeping 11248620 word types
2017-10-25 10:13:47,377 [MainThread  ] [INFO ]  PROGRESS: at sentence #91740000, processed 963060580 words, keeping 11249610 word types
2017-10-25 10:13:48,576 [MainThread  ] [INFO ]  PROGRESS: at sentence #91750000, processed 963172112 words, keeping 11250547 word types
2017-10-25 10:13:49,733 [MainThread  ] [INFO ]  PROGRESS: at sentence #91760000, processed 963279403 words, keeping 11250929 word types
2017-10-25 10:13:50,857 [MainThread  ] [INFO ]  

2017-10-25 10:14:53,104 [MainThread  ] [INFO ]  PROGRESS: at sentence #92310000, processed 969072419 words, keeping 11298490 word types
2017-10-25 10:14:54,245 [MainThread  ] [INFO ]  PROGRESS: at sentence #92320000, processed 969177462 words, keeping 11298968 word types
2017-10-25 10:14:55,369 [MainThread  ] [INFO ]  PROGRESS: at sentence #92330000, processed 969280523 words, keeping 11299579 word types
2017-10-25 10:14:56,572 [MainThread  ] [INFO ]  PROGRESS: at sentence #92340000, processed 969393066 words, keeping 11300249 word types
2017-10-25 10:14:57,730 [MainThread  ] [INFO ]  PROGRESS: at sentence #92350000, processed 969500073 words, keeping 11300837 word types
2017-10-25 10:14:58,852 [MainThread  ] [INFO ]  PROGRESS: at sentence #92360000, processed 969602367 words, keeping 11301594 word types
2017-10-25 10:15:00,011 [MainThread  ] [INFO ]  PROGRESS: at sentence #92370000, processed 969709561 words, keeping 11302183 word types
2017-10-25 10:15:01,164 [MainThread  ] [INFO ]  

2017-10-25 10:16:03,338 [MainThread  ] [INFO ]  PROGRESS: at sentence #92920000, processed 975516101 words, keeping 11346494 word types
2017-10-25 10:16:04,504 [MainThread  ] [INFO ]  PROGRESS: at sentence #92930000, processed 975625433 words, keeping 11347220 word types
2017-10-25 10:16:05,653 [MainThread  ] [INFO ]  PROGRESS: at sentence #92940000, processed 975729114 words, keeping 11347700 word types
2017-10-25 10:16:06,803 [MainThread  ] [INFO ]  PROGRESS: at sentence #92950000, processed 975836909 words, keeping 11348335 word types
2017-10-25 10:16:07,945 [MainThread  ] [INFO ]  PROGRESS: at sentence #92960000, processed 975942778 words, keeping 11348966 word types
2017-10-25 10:16:09,075 [MainThread  ] [INFO ]  PROGRESS: at sentence #92970000, processed 976045760 words, keeping 11349644 word types
2017-10-25 10:16:10,233 [MainThread  ] [INFO ]  PROGRESS: at sentence #92980000, processed 976150844 words, keeping 11350422 word types
2017-10-25 10:16:11,409 [MainThread  ] [INFO ]  

2017-10-25 10:17:13,246 [MainThread  ] [INFO ]  PROGRESS: at sentence #93530000, processed 981902930 words, keeping 11395550 word types
2017-10-25 10:17:14,395 [MainThread  ] [INFO ]  PROGRESS: at sentence #93540000, processed 982008408 words, keeping 11396454 word types
2017-10-25 10:17:15,531 [MainThread  ] [INFO ]  PROGRESS: at sentence #93550000, processed 982113051 words, keeping 11396955 word types
2017-10-25 10:17:16,626 [MainThread  ] [INFO ]  PROGRESS: at sentence #93560000, processed 982213180 words, keeping 11397529 word types
2017-10-25 10:17:17,767 [MainThread  ] [INFO ]  PROGRESS: at sentence #93570000, processed 982319667 words, keeping 11398012 word types
2017-10-25 10:17:18,929 [MainThread  ] [INFO ]  PROGRESS: at sentence #93580000, processed 982427146 words, keeping 11398530 word types
2017-10-25 10:17:20,089 [MainThread  ] [INFO ]  PROGRESS: at sentence #93590000, processed 982535545 words, keeping 11399156 word types
2017-10-25 10:17:21,234 [MainThread  ] [INFO ]  

In [8]:
model.save('../data/lingvo/w2v_300_sg_5_w10_trigram')

2017-10-24 16:58:41,140 [MainThread  ] [INFO ]  saving Word2Vec object under ../data/lingvo/w2v_300_sg_5_w10_trigram, separately None
2017-10-24 16:58:41,141 [MainThread  ] [INFO ]  not storing attribute syn0norm
2017-10-24 16:58:41,142 [MainThread  ] [INFO ]  storing np array 'syn0' to ../data/lingvo/w2v_300_sg_5_w10_trigram.wv.syn0.npy
2017-10-24 16:58:59,723 [MainThread  ] [INFO ]  storing np array 'syn1neg' to ../data/lingvo/w2v_300_sg_5_w10_trigram.syn1neg.npy
2017-10-24 16:59:18,031 [MainThread  ] [INFO ]  not storing attribute cum_table
2017-10-24 16:59:46,558 [MainThread  ] [INFO ]  saved ../data/lingvo/w2v_300_sg_5_w10_trigram


In [11]:
for w ,s in model.wv.most_similar(u'стол'):
    print('%s %s' % (w,s))

нпм 0.297611892223
выполпеп 0.277081847191
130g 0.268700033426
балаковский_аэс 0.257275521755
(125/25 0.256969988346
оранжевый_спектросмещать 0.253416627645
замена_к32е. 0.252790808678
сверхразведение_матричный 0.250886440277
цикло-(D-Val-NMeArg-Gly-Asp-Mamb(5-aca 0.249853163958
аэро_космоснимок 0.24928149581


# Save for Misha

In [5]:
trigram_ph = gensim.models.phrases.Phraser.load('../data/lingvo/trigram_ph')

2017-10-24 09:33:56,826 [MainThread  ] [INFO ]  loading Phraser object from ../data/lingvo/trigram_ph
2017-10-24 09:33:58,577 [MainThread  ] [INFO ]  loaded ../data/lingvo/trigram_ph


In [29]:
c = 0
l = []
for k,v in tqdm_notebook(trigram_ph.phrasegrams.items()):
    if v[0] > 10 and v[1] > 30:
        the_string = '_'.join(k)
        if re.search('[а-яА-Яa-zA-Z]', k[0]) and re.search('[а-яА-Яa-zA-Z]', k[1]):
            c+=1
            l.append([the_string, v[0], v[1]])
#         print('%s_%s %s' % (k[0], k[1], v))
        
df = pd.DataFrame(l)

In [ ]:
# df.sort_values(by=1, ascending=False)

In [30]:
len(df[0].unique())

1279338

In [34]:
with GzipFile('../data/collocations_bigram_all.txt.gz', 'w') as f:
    for tag in sorted(df[0].unique()):
        f.write(tag + '\n')

In [2]:
import xml.etree.ElementTree as ET

tree = ET.parse('../data/TestDocumentsResult.xml')
root = tree.getroot()

In [30]:
gold = {}
for child in root:
    sims = []
    for similars in child:
        for sim in similars:
            sims.append(sim.attrib['docId'])
    gold[child.attrib['docId']] = sims